In [2]:
from sqlalchemy import MetaData, Table, select, create_engine, desc

engine = create_engine('sqlite:///census.sqlite')
connection = engine.connect()  
metadata = MetaData()
census = Table('census', metadata, autoload = True, autoload_with = engine)

# Calculating Value

In [3]:
query = select([census.columns.sex,
                (census.columns.pop2008 - 
                 census.columns.pop2000).label('pop_change')])
query = query.group_by(census.columns.age)          # Grouping
query = query.order_by(desc('pop_change'))          # Ordering
query = query.limit(5)                              # limiting the query
results = connection.execute(query).fetchall()
print(results)
print(results[0].keys())

[('M', 25201), ('M', 23503), ('M', 21716), ('M', 19677), ('M', 19526)]
RMKeyView(['sex', 'pop_change'])


In [4]:
from sqlalchemy import func
from num2words import num2words
query = select([func.sum(census.columns.pop2008)])

results= connection.execute(query).scalar()
print(results)
print(num2words(results))

302876613
three hundred and two million, eight hundred and seventy-six thousand, six hundred and thirteen


In [5]:
from num2words import num2words
from sqlalchemy import func, or_

query = select([func.sum(census.columns.pop2008)])
query = query.where(census.columns.state == 'California')

results= connection.execute(query).scalar()
print(results)
print(num2words(results))

36609002
thirty-six million, six hundred and nine thousand and two


In [6]:
from num2words import num2words
from sqlalchemy import func, or_

query = select([func.sum(census.columns.pop2008)])
query = query.where(
                    or_(census.columns.state == 'California',
                        census.columns.state == 'New York'))
results= connection.execute(query).scalar()
print(results)
print(num2words(results))

56074161
fifty-six million, seventy-four thousand, one hundred and sixty-one


In [7]:
from num2words import num2words
from sqlalchemy import func, and_

query = select([func.sum(census.columns.pop2008)])
query = query.where(
                    and_(census.columns.state == 'New York', #Filtering (and) statement
                        census.columns.sex == 'M'))
results = connection.execute(query).scalar()
print(results)
print(num2words(results))

9439935
nine million, four hundred and thirty-nine thousand, nine hundred and thirty-five


### Case Statement

In [8]:
from sqlalchemy import case, func
query = select([
        func.sum(
            case([                                 # We use this if the where statement cannot perform
                (census.columns.state == 'New York',
                 census.columns.pop2008)
            ], else_=0))
])

results = connection.execute(query).scalar()
print(results)
print(num2words(results))

19465159
nineteen million, four hundred and sixty-five thousand, one hundred and fifty-nine


The previous one will make the same result as this : 

In [9]:
from num2words import num2words
from sqlalchemy import func

query = select([func.sum(census.columns.pop2008)])
query = query.where(census.columns.state == 'New York')

results= connection.execute(query).scalar()
print(results)
print(num2words(results))

19465159
nineteen million, four hundred and sixty-five thousand, one hundred and fifty-nine


### Cast Statement

The CAST() function converts a value (of any type) into a specified datatype.

In [10]:
from num2words import num2words
from sqlalchemy import func, case, cast, Float

query = select([
                (func.sum(
                case([
                    (census.columns.state == 'New York',
                     census.columns.pop2008)
                ], else_=0)) / 
                cast(func.sum(census.columns.pop2008), Float) * 100).label('ny_percent')])


results= connection.execute(query).scalar()
print(results)
print(num2words(results))

6.426761976501632
six point four two six seven six one nine seven six five zero one six three two


# SQL relationship

In [13]:
from sqlalchemy import MetaData, Table, select

metadata = MetaData()
census = Table('census', metadata, autoload = True, autoload_with = engine)
query = select([census]) # selecting all of the column in census table
results = connection.execute(query).fetchmany(10)
results

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111),
 ('Illinois', 'M', 5, 93894, 89802),
 ('Illinois', 'M', 6, 93676, 88931),
 ('Illinois', 'M', 7, 94818, 90940),
 ('Illinois', 'M', 8, 95035, 86943),
 ('Illinois', 'M', 9, 96436, 86055)]

In [14]:
from sqlalchemy import MetaData, Table, select

metadata = MetaData()
state_fact = Table('state_fact', metadata, autoload = True, autoload_with = engine)
query = select([state_fact]) # selecting all of the column in state_fact table
results = connection.execute(query).fetchmany(10)
print(results)
print(results[0].keys())
print(results[0].name) # watch this line

[('13', 'Illinois', 'IL', 'USA', 'state', '10', 'current', 'occupied', '', '17', 'Ill.', 'V', '2', 'Midwest', '3', 'East North Central', '7'), ('30', 'New Jersey', 'NJ', 'USA', 'state', '10', 'current', 'occupied', '', '34', 'N.J.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '3'), ('34', 'North Dakota', 'ND', 'USA', 'state', '10', 'current', 'occupied', '', '38', 'N.D.', 'VIII', '2', 'Midwest', '4', 'West North Central', '8'), ('37', 'Oregon', 'OR', 'USA', 'state', '10', 'current', 'occupied', '', '41', 'Ore.', 'X', '4', 'West', '9', 'Pacific', '9'), ('51', 'Washington DC', 'DC', 'USA', 'capitol', '10', 'current', 'occupied', '', '11', '', 'III', '3', 'South', '5', 'South Atlantic', 'D.C.'), ('49', 'Wisconsin', 'WI', 'USA', 'state', '10', 'current', 'occupied', '', '55', 'Wis.', 'V', '2', 'Midwest', '3', 'East North Central', '7'), ('3', 'Arizona', 'AZ', 'USA', 'state', '10', 'current', 'occupied', '', '4', 'Ariz.', 'IX', '4', 'West', '8', 'Mountain', '9'), ('4', 'Arkansas', 'AR', 'U

In [15]:
from sqlalchemy import MetaData, Table, select

metadata = MetaData()
census = Table('census', metadata, autoload = True, autoload_with = engine)
query = select([census]) # selecting all of the column in census table
results = connection.execute(query).fetchmany(10)
print(results)
print(results[0].keys())
print(results[0].state)  # watch this line

[('Illinois', 'M', 0, 89600, 95012), ('Illinois', 'M', 1, 88445, 91829), ('Illinois', 'M', 2, 88729, 89547), ('Illinois', 'M', 3, 88868, 90037), ('Illinois', 'M', 4, 91947, 91111), ('Illinois', 'M', 5, 93894, 89802), ('Illinois', 'M', 6, 93676, 88931), ('Illinois', 'M', 7, 94818, 90940), ('Illinois', 'M', 8, 95035, 86943), ('Illinois', 'M', 9, 96436, 86055)]
RMKeyView(['state', 'sex', 'age', 'pop2000', 'pop2008'])
Illinois


### Auto Join

In [16]:
query = select([census.columns.pop2008, state_fact.columns.abbreviation])
results = connection.execute(query).fetchmany(10)
print(results)
print(results[0].keys())
print(results[0])

[(95012, 'IL'), (95012, 'NJ'), (95012, 'ND'), (95012, 'OR'), (95012, 'DC'), (95012, 'WI'), (95012, 'AZ'), (95012, 'AR'), (95012, 'CO'), (95012, 'HI')]
RMKeyView(['pop2008', 'abbreviation'])
(95012, 'IL')


C:\Users\muham\AppData\Local\Temp\ipykernel_19568\1977118855.py:2: SAWarning: SELECT statement has a cartesian product between FROM element(s) "state_fact" and FROM element "census".  Apply join condition(s) between each element to resolve.
  results = connection.execute(query).fetchmany(10)


### Join 

In [22]:
from sqlalchemy import MetaData, Table, select

metadata = MetaData()
census = Table('census', metadata, autoload = True, autoload_with = engine)
query = select([census]) # selecting all of the column in census table
results = connection.execute(query).fetchmany(10)
results[0].keys()

RMKeyView(['state', 'sex', 'age', 'pop2000', 'pop2008'])

In [24]:
from sqlalchemy import MetaData, Table, select

metadata = MetaData()
state_fact = Table('state_fact', metadata, autoload = True, autoload_with = engine)
query = select([state_fact]) # selecting all of the column in state_fact table
results = connection.execute(query).fetchmany(10)

print(results[0].keys())


RMKeyView(['id', 'name', 'abbreviation', 'country', 'type', 'sort', 'status', 'occupied', 'notes', 'fips_state', 'assoc_press', 'standard_federal_region', 'census_region', 'census_region_name', 'census_division', 'census_division_name', 'circuit_court'])


In [26]:

# JOINING THE TABLE

query = select([func.sum(census.columns.pop2000)])
query = query.select_from(
                    census.join(state_fact, census.columns.state == state_fact.columns.name)) # matching the column
query = query.where(state_fact.columns.census_division_name == 'East South Central') #With filter, but it actually possible without filter top
results = connection.execute(query).scalar()
print(results)

16982311


# Hierarchical tables